In [8]:
import sys
!{sys.executable} -m pip install geocoder

print('Packages installed.')

Packages installed.


In [9]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


Scrape the postal codes of Canada using Beautiful Soap

In [25]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(wiki_url) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

Data Collection

In [26]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned\n':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1

In [27]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

Using the Geocoder API

In [28]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [29]:
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A\n,North York\n,Parkwoods\n,43.75188,-79.33036
1,M4A\n,North York\n,Victoria Village\n,43.73042,-79.31282
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.65514,-79.36265
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.72321,-79.45141
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.66449,-79.39302
...,...,...,...,...,...
97,M5X\n,Downtown Toronto\n,"First Canadian Place, Underground city\n",43.64828,-79.38146
98,M8X\n,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n",43.65369,-79.51112
99,M4Y\n,Downtown Toronto\n,Church and Wellesley\n,43.66659,-79.38130
100,M7Y\n,East Toronto\n,"Business reply mail Processing Centre, South C...",43.64869,-79.38544


In [30]:
neighbors.shape

(102, 5)

In [31]:
VERSION = '20180605' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     43.67635739999999, 
#     79.2930312, 
#     radius, 
#     LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore?&client_id=RI5QCZXECCSAFX30PCWF2BGNC0VXSPUKI20IJZ0VTDGNFQZD&client_secret=TYDVJ3USKRG3WL41LDZHM5BTYB51WHC3MR00NCBPSE2SS3FD&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

results = requests.get(url).json()
results

Latitude and longitude values of Parkwoods
 are 43.75188000000003, -79.33035999999998.


{'meta': {'code': 200, 'requestId': '5f00995b1116f15047b8bc41'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

In [32]:
# From the Foursquare lab in the previous module, we know that all the information is in the items key. 
# Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Seaspray Restaurant,Asian Restaurant,43.678888,-79.298167


In [35]:
print('{} venues that Foursquare returned.'.format(nearby_venues.shape[0]))

5 venues that Foursquare returned.


In [36]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

Parkwoods

Victoria Village

Regent Park, Harbourfront

Lawrence Manor, Lawrence Heights

Queen's Park, Ontario Provincial Government

Islington Avenue, Humber Valley Village

Malvern, Rouge

Don Mills

Parkview Hill, Woodbine Gardens

Garden District, Ryerson

Glencairn

West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale

Rouge Hill, Port Union, Highland Creek

Don Mills

Woodbine Heights

St. James Town

Humewood-Cedarvale

Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Guildwood, Morningside, West Hill

The Beaches

Berczy Park

Caledonia-Fairbanks

Woburn

Leaside

Central Bay Street

Christie

Cedarbrae

Hillcrest Village

Bathurst Manor, Wilson Heights, Downsview North

Thorncliffe Park

Richmond, Adelaide, King

Dufferin, Dovercourt Village

Scarborough Village

Fairview, Henry Farm, Oriole

Northwood Park, York University

East Toronto, Broadview North (Old East York)

Harbourfront East, Union Station, Toronto Islands

Little Portugal, Trinit

In [38]:
# check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(510, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods\n,43.75188,-79.33036,Glen Manor Ravine,43.676821,-79.293942,Trail
1,Parkwoods\n,43.75188,-79.33036,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,Parkwoods\n,43.75188,-79.33036,Grover Pub and Grub,43.679181,-79.297215,Pub
3,Parkwoods\n,43.75188,-79.33036,Upper Beaches,43.680563,-79.292869,Neighborhood
4,Parkwoods\n,43.75188,-79.33036,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


In [39]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt\n,5,5,5,5,5,5
"Alderwood, Long Branch\n",5,5,5,5,5,5
"Bathurst Manor, Wilson Heights, Downsview North\n",5,5,5,5,5,5
Bayview Village\n,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East\n",5,5,5,5,5,5
...,...,...,...,...,...,...
"Willowdale, Willowdale West\n",5,5,5,5,5,5
Woburn\n,5,5,5,5,5,5
Woodbine Heights\n,5,5,5,5,5,5


In [40]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 5 uniques categories.


In [41]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Trail,Asian Restaurant,Health Food Store,Neighborhood,Pub
0,1,0,0,Parkwoods\n,0
1,0,0,1,Parkwoods\n,0
2,0,0,0,Parkwoods\n,1
3,0,0,0,Parkwoods\n,0
4,0,1,0,Parkwoods\n,0


In [42]:
# examine the new dataframe size.

toronto_onehot.shape

(510, 5)

In [43]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Trail,Asian Restaurant,Health Food Store,Pub
0,Agincourt\n,0.2,0.2,0.2,0.2
1,"Alderwood, Long Branch\n",0.2,0.2,0.2,0.2
2,"Bathurst Manor, Wilson Heights, Downsview North\n",0.2,0.2,0.2,0.2
3,Bayview Village\n,0.2,0.2,0.2,0.2
4,"Bedford Park, Lawrence Manor East\n",0.2,0.2,0.2,0.2
...,...,...,...,...,...
93,"Willowdale, Willowdale West\n",0.2,0.2,0.2,0.2
94,Woburn\n,0.2,0.2,0.2,0.2
95,Woodbine Heights\n,0.2,0.2,0.2,0.2
96,York Mills West\n,0.2,0.2,0.2,0.2


In [44]:
# confirm the new size

toronto_grouped.shape

(98, 5)

In [45]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Alderwood, Long Branch
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bathurst Manor, Wilson Heights, Downsview North
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bayview Village
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bedford Park, Lawrence Manor East
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Berczy Park
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3         

In [46]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt\n,Pub,Health Food Store,Asian Restaurant
1,"Alderwood, Long Branch\n",Pub,Health Food Store,Asian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North\n",Pub,Health Food Store,Asian Restaurant
3,Bayview Village\n,Pub,Health Food Store,Asian Restaurant
4,"Bedford Park, Lawrence Manor East\n",Pub,Health Food Store,Asian Restaurant


In [49]:
# Run k-means to cluster the neighborhood into 5 clusters.
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [50]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M3A\n,North York\n,Parkwoods\n,43.75188,-79.33036,0,Pub,Health Food Store,Asian Restaurant
1,M4A\n,North York\n,Victoria Village\n,43.73042,-79.31282,0,Pub,Health Food Store,Asian Restaurant
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.65514,-79.36265,0,Pub,Health Food Store,Asian Restaurant
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.72321,-79.45141,0,Pub,Health Food Store,Asian Restaurant
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.66449,-79.39302,0,Pub,Health Food Store,Asian Restaurant


In [53]:
# Finally, let's visualize the resulting clusters
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [54]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,North York\n,0,Pub,Health Food Store,Asian Restaurant
1,North York\n,0,Pub,Health Food Store,Asian Restaurant
2,Downtown Toronto\n,0,Pub,Health Food Store,Asian Restaurant
3,North York\n,0,Pub,Health Food Store,Asian Restaurant
4,Downtown Toronto\n,0,Pub,Health Food Store,Asian Restaurant
...,...,...,...,...,...
97,Downtown Toronto\n,0,Pub,Health Food Store,Asian Restaurant
98,Etobicoke\n,0,Pub,Health Food Store,Asian Restaurant
99,Downtown Toronto\n,0,Pub,Health Food Store,Asian Restaurant
100,East Toronto\n,0,Pub,Health Food Store,Asian Restaurant
